In [ ]:
from IPython.display import Markdown, display

def show_md(text: str):
    display(Markdown(text if text.strip() else "_<vacío>_"))

async def ask_naive(q: str):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    return await rag.aquery(prompt, QueryParam(mode="naive"))

async def ask_global(q: str):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    return await rag.aquery(prompt, QueryParam(mode="global"))

async def ask_llm_only(q: str, temperature: float = 0.6, max_tokens: int = 160):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    return await deepseepk_model_if_cache(prompt, temperature=temperature, max_tokens=max_tokens)

async def compare(q: str, do_llm: bool = True):
    n = await ask_naive(q)
    g = await ask_global(q)
    show_md(f"### ❓ {q}\n\n**🧩 Naive RAG**\n\n{n}\n\n**🌐 GraphRAG (Global)**\n\n{g}")
    if do_llm:
        l = await ask_llm_only(q)
        show_md(f"**🧠 Puro LLM**\n\n{l}")

# Ejemplo:
# await compare("¿Quiénes son los protagonistas?")

In [11]:
await compare("¿Quiénes son los protagonistas?")

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.72it/s]


TypeError: 'NoneType' object is not subscriptable

In [8]:
# === Ejemplo de comparación ===
question = "Quienes son los protagonista?"

ans_naive  = await ask_naive(question)
# ans_global = await ask_global(question)
# ans_llm    = await ask_llm_only(question, temperature=0.5, max_tokens=180)

print("Pregunta:");   show_md(question)
print("NAIVE RAG:\n");   show_md(ans_naive)
# print("\nGRAPH RAG (GLOBAL):\n"); show_md(ans_global)
# print("\nPURO LLM:\n");    show_md(ans_llm)

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.43it/s]


TypeError: 'NoneType' object is not subscriptable

In [13]:

import json, os
from pathlib import Path
from examples.using_deepseek_as_llm import (
    deepseepk_model_if_cache,   # LLM ya listo y con cache opcional
    local_st_embeddings,
    WORKING_DIR,
)

print("WORKING_DIR:", WORKING_DIR)
print("enable_naive_rag:", rag.enable_naive_rag)
print("chunks_vdb is None? ->", rag.chunks_vdb is None)

def size(p): 
    return Path(p).stat().st_size if Path(p).exists() else 0

vdb_path = f"{WORKING_DIR}/vdb_chunks.json"
kv_path  = f"{WORKING_DIR}/kv_store_text_chunks.json"

print("vdb_chunks.json bytes:", size(vdb_path))
print("kv_store_text_chunks.json bytes:", size(kv_path))

if size(kv_path):
    with open(kv_path, encoding="utf-8") as f:
        d = json.load(f)
    print("KV chunks:", len(d))
    print("Ejemplo de IDs:", list(d.keys())[:3])

WORKING_DIR: ./nano_graphrag_cache_deepseek_TEST
enable_naive_rag: True
chunks_vdb is None? -> False
vdb_chunks.json bytes: 296428
kv_store_text_chunks.json bytes: 638909
KV chunks: 99
Ejemplo de IDs: ['chunk-21453a7fcc3285739b8573a25e0e6893', 'chunk-c0949c6a647fd1346bc913f8f9e13fcc', 'chunk-97895642add93678aae24271f6d4f93e']
